In [1]:
import os
import glob
import mne
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import clevercsv
import json

data_root = '../Data/0000000000000'

def time2time(timein):
    return '{:02d}:{:02d}'.format(*divmod(timein, 60))

def time_str2float(in_date):
    return float(in_date.replace(':','.'))

In [2]:
fields = pd.read_csv(f'{data_root}/STR_signals.txt')

# read the STR resumee edf file converted with edf2ascii.exe
data = pd.read_csv(f'{data_root}/STR_data.txt')
data.dropna(inplace=True)
data_str_f = data.copy()
data_str_f.drop(columns='Time', inplace= True)

# update the column name
colum_names =[]
for element in fields['Label'].values.tolist():
    colum_names.append(element.strip())
    
data_str_f.columns=colum_names

# add human readable date format 
time_ref = datetime.datetime(1970,1,1)
date = []
for element in data_str_f['Date'].tolist():
    date.append((time_ref+datetime.timedelta(days=element)).isoformat().split('T')[0])

data_str_f.insert(1, 'R_Date', date, True)

In [3]:
data = pd.read_csv(f'{data_root}/STR_data.txt')
data.head(20)

,Time,1,2,3,4,5,6,7,8,9,...,71,72,73,74,75,76,77,78,79,80
0,0.0,19279.0,631.0,1081.0,2.0,450.0,450.0,14606.0,1.0,2.0,...,0.2,0.1,0.1,0.0,0.0,0.0,0.0,0.0,0.0,-26584.0
1,8640.0,NaN,-1.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,17280.0,NaN,-1.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,25920.0,NaN,-1.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,34560.0,NaN,-1.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,43200.0,NaN,-1.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,51840.0,NaN,-1.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,60480.0,NaN,-1.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,69120.0,NaN,-1.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,77760.0,NaN,-1.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
fields.Label.to_list()

['Date            ',
 'MaskOn          ',
 'MaskOff         ',
 'MaskEvents      ',
 'Duration        ',
 'OnDuration      ',
 'PatientHours    ',
 'Mode            ',
 'S.RampEnable    ',
 'S.RampTime      ',
 'S.C.StartPress  ',
 'S.C.Press       ',
 'S.EPR.ClinEnable',
 'S.EPR.EPREnable ',
 'S.EPR.Level     ',
 'S.EPR.EPRType   ',
 'S.AS.Comfort    ',
 'S.AS.StartPress ',
 'S.AS.MaxPress   ',
 'S.AS.MinPress   ',
 'S.SmartStart    ',
 'S.PtAccess      ',
 'S.ABFilter      ',
 'S.Mask          ',
 'S.Tube          ',
 'S.ClimateControl',
 'S.HumEnable     ',
 'S.HumLevel      ',
 'S.TempEnable    ',
 'S.Temp          ',
 'HeatedTube      ',
 'Humidifier      ',
 'BlowPress.95    ',
 'BlowPress.5     ',
 'Flow.95         ',
 'Flow.5          ',
 'BlowFlow.50     ',
 'AmbHumidity.50  ',
 'HumTemp.50      ',
 'HTubeTemp.50    ',
 'HTubePow.50     ',
 'HumPow.50       ',
 'SpO2.50         ',
 'SpO2.95         ',
 'SpO2.Max        ',
 'SpO2Thresh      ',
 'MaskPress.50    ',
 'MaskPress.9

In [5]:
i = 0
for elements in colum_names:
    print(i, elements)
    i=i+1

0 Date
1 MaskOn
2 MaskOff
3 MaskEvents
4 Duration
5 OnDuration
6 PatientHours
7 Mode
8 S.RampEnable
9 S.RampTime
10 S.C.StartPress
11 S.C.Press
12 S.EPR.ClinEnable
13 S.EPR.EPREnable
14 S.EPR.Level
15 S.EPR.EPRType
16 S.AS.Comfort
17 S.AS.StartPress
18 S.AS.MaxPress
19 S.AS.MinPress
20 S.SmartStart
21 S.PtAccess
22 S.ABFilter
23 S.Mask
24 S.Tube
25 S.ClimateControl
26 S.HumEnable
27 S.HumLevel
28 S.TempEnable
29 S.Temp
30 HeatedTube
31 Humidifier
32 BlowPress.95
33 BlowPress.5
34 Flow.95
35 Flow.5
36 BlowFlow.50
37 AmbHumidity.50
38 HumTemp.50
39 HTubeTemp.50
40 HTubePow.50
41 HumPow.50
42 SpO2.50
43 SpO2.95
44 SpO2.Max
45 SpO2Thresh
46 MaskPress.50
47 MaskPress.95
48 MaskPress.Max
49 TgtIPAP.50
50 TgtIPAP.95
51 TgtIPAP.Max
52 TgtEPAP.50
53 TgtEPAP.95
54 TgtEPAP.Max
55 Leak.50
56 Leak.95
57 Leak.70
58 Leak.Max
59 MinVent.50
60 MinVent.95
61 MinVent.Max
62 RespRate.50
63 RespRate.95
64 RespRate.Max
65 TidVol.50
66 TidVol.95
67 TidVol.Max
68 AHI
69 HI
70 AI
71 OAI
72 CAI
73 UAI
74 CSR
75

In [ ]:
#periode = ['2023-10-14','2023-10-17']
#start_periode = data_str.index[data_str['R_Date'] ==periode[0]].tolist() # detect concerned row
#end_periode = data_str.index[data_str['R_Date'] ==periode[1]].tolist() # detect concerned row

# Check the number of hours per section duration etc for one day 

In [ ]:
# Convert Gaetan time format to human time
#Oui c'est'' maskOn et maskOff 10 valeurs maxi par jour en minute écoulée de depuis date (en jour) à midi

mask_on = 695
mask_off = 1086

init = time2time(mask_on)
finit = time2time(mask_off)
if int(finit.split(':')[0])>12:
    finitfinal = f'{int(finit.split(":")[0])-12}:{int(finit.split(":")[1])}'
else:
    finitfinal = 'Not midnight passed'
    
duration = datetime.datetime.strptime(finit, "%H:%M") - datetime.datetime.strptime(init, "%H:%M")
print(f'from {init} to {finit} ---> {finitfinal} am')
print(f'duration {duration}')

In [ ]:
duration = 628
'{:02d}:{:02d}'.format(*divmod(duration, 60))

# Create donnes synthese for a periode using only STR information

In [7]:
queries_columns = ['Date', 'R_Date', 'MaskOn', 'MaskOff', 'MaskEvents', 'Duration',
                   'AHI', 'HI', 'AI', 'OAI', 'CAI','UAI',
                   'Leak.50', 'Leak.95', 'Leak.Max',  'MaskPress.50', 'MaskPress.95', 'MaskPress.Max']


# create the points to plot and the respective date format
periode = ['2023-10-03','2023-10-07']
id_i=int(data_str_f.index[data_str_f['R_Date']==periode[0]].tolist()[0]/10)
id_f=int(data_str_f.index[data_str_f['R_Date']==periode[1]].tolist()[0]/10)
data_str = data_str_f[queries_columns].iloc[id_i:id_f+1].copy()

Duration = list(map(int, data_str['Duration'].to_list()))
R_Duration =  list(map(time2time, Duration))
R_Duration = list(map(time_str2float, R_Duration))

data_str.insert(4, 'R_Duration', R_Duration, True )
data_str

,Date,R_Date,MaskOn,MaskOff,R_Duration,MaskEvents,Duration,AHI,HI,AI,OAI,CAI,UAI,Leak.50,Leak.95,Leak.Max,MaskPress.50,MaskPress.95,MaskPress.Max
3540,19633.0,2023-10-03,559.0,1088.0,8.49,2.0,529.0,0.6,0.1,0.5,0.5,0.0,0.0,0.0,0.12,0.38,8.52,9.96,9.96
3550,19634.0,2023-10-04,674.0,1088.0,6.54,2.0,414.0,1.0,0.1,0.8,0.8,0.0,0.0,0.0,0.42,0.52,8.16,9.96,9.96
3560,19635.0,2023-10-05,675.0,1087.0,6.52,2.0,412.0,1.6,0.4,1.1,0.8,0.2,0.0,0.0,0.32,0.56,8.76,9.96,9.96
3570,19636.0,2023-10-06,622.0,1162.0,9.00,2.0,540.0,0.5,0.2,0.3,0.3,0.0,0.0,0.0,0.30,0.80,9.00,9.96,9.96
3580,19637.0,2023-10-07,615.0,1168.0,9.13,2.0,553.0,0.3,0.1,0.2,0.2,0.0,0.0,0.0,0.34,0.58,8.64,9.96,9.96


In [ ]:
fig, ax = plt.subplots(5,1,figsize=(16,10))

#ax[0].bar(data_str['Date'], data_str['Duration'], bottom=change_time_format(np.array(data_str['Init_time'])))
ax[0].bar(data_str['Date'], data_str['R_Duration'])
#ax[0].set_xticks(data_str['Date'])
ax[0].set_ylabel('time (H)')
#ax[4].set_ylim(0, 32)
#the_ticks = ax[4].get_yticks()
#new_ticks =correc_yticks(the_ticks)
#ax[4].set_yticks(the_ticks, new_ticks)
ax[0].set_xticks([])
#ax[4].grid()

ax[1].bar(data_str['Date'], data_str['R_Duration'], color='olivedrab')
#ax[3].bar(data_str['Date'], data_str['fluites_max']*3, color='lightblue') # invented data[non CRS on EDF or CSV]
ax[1].axhline(y = 4, color = 'r', linestyle = '-') 
ax[1].set_ylabel('utilization (H)',fontsize=12)
ax[1].set_xticks([])
ax[1].legend(["4H", "utilisation", "temp_RCS"], loc="lower right", bbox_to_anchor=(-0.05, 0.1))

ax[4].bar(data_str['Date'], data_str['MaskPress.Max'], color='gray' )
ax[4].bar(data_str['Date'], data_str['MaskPress.95'], color='lightsteelblue' )
ax[4].bar(data_str['Date'], data_str['MaskPress.50'], color='lightskyblue' )
ax[4].set_ylabel('Pression (cmH20)', fontsize=12)
#ax[4].set_xticks([])
ax[4].set_xticks(data_str['Date'])
ax[4].legend(["P_max", "P_95", "P_mean"], loc="lower right",  bbox_to_anchor=(-0.05, 0.1))

ax[3].bar(data_str['Date'], data_str['Leak.Max'], color='darkgoldenrod' )
ax[3].bar(data_str['Date'], data_str['Leak.95'], color='goldenrod' )
ax[3].bar(data_str['Date'], data_str['Leak.50'], color='gold' )
ax[3].set_ylabel('Fluites (l/min)', fontsize=12)
ax[3].set_xticks([])
ax[3].legend(["F_max", "F_95", "F_mean"], loc="lower right", bbox_to_anchor=(-0.05, 0.1))

ax[2].bar(data_str['Date'], data_str['OAI']+data_str['HI']+data_str['CAI'], color='indianred')    # invented data[non IAC on csv]
ax[2].bar(data_str['Date'], data_str['HI']+data_str['CAI'], color='mistyrose')
ax[2].bar(data_str['Date'], data_str['CAI'], color='silver')
ax[2].set_ylabel('Events', fontsize=12)
ax[2].set_xticks([])
ax[2].legend(['IAO', 'IH', 'IAC'], loc="lower right",  bbox_to_anchor=(-0.05, 0.1))
ax[2].grid()

for i in range(len(data_str['R_Date'].unique())):
    ax[0].text(data_str['Date'].unique()[i], 11, data_str['R_Date'].unique()[i], rotation=45, fontsize=12)

#fig.savefig('Graph_synt.png', dpi=1000)

In [ ]:
fig, ax = plt.subplots(2,1,figsize=(16,10))

ax[0].bar(data_str['Date'], data_str['OAI']+data_str['HI']+data_str['CAI'], color='indianred')    # invented data[non IAC on csv]
ax[0].bar(data_str['Date'], data_str['HI']+data_str['CAI'], color='mistyrose')
ax[0].bar(data_str['Date'], data_str['CAI'], color='silver')
ax[0].set_ylabel('Events', fontsize=12)
ax[0].set_xticks([])
ax[0].legend(['IAO', 'IH', 'IAC'], loc="lower right",  bbox_to_anchor=(-0.05, 0.1))
ax[0].grid()

ax[1].bar(data_str['Date'], data_str['R_Duration'], color='darkgreen')
#ax[3].bar(data_str['Date'], data_str['fluites_max']*3, color='lightblue') # invented data[non CRS on EDF or CSV]
ax[1].axhline(y = 4, color = 'r', linestyle = '-') 
ax[1].set_ylabel('utilization (H)',fontsize=12)
ax[1].set_xticks([])
ax[1].legend(["4H", "utilisation", "temp_RCS"], loc="lower right", bbox_to_anchor=(-0.05, 0.1))

for i in range(len(data_str['R_Date'].unique())):
    ax[0].text(data_str['Date'].unique()[i], 1.6, data_str['R_Date'].unique()[i], rotation=45, fontsize=12)

In [ ]:
np.mean(data_str['R_Duration'].to_list())

# Working with the EDF file donnes detailles

In [ ]:
folder = '20231001'
#qms_edf = mne.io.read_raw_edf(glob.glob(f'{data_root}/{folder}/*BRP.edf')[0], verbose='error')
ts_edf = mne.io.read_raw_edf(glob.glob(f'{data_root}/{folder}/*PLD.edf')[0], verbose='error')
evt = pd.read_csv(glob.glob(f'{data_root}/{folder}/*EVT.csv')[0], skiprows=[0], sep = ';', names=list('ABCD'))

In [ ]:
#data_edf = mne.io.read_raw_edf(f'{data_root}/STR.edf')
#data_edf.__dict__
#A.mapping.keys()
#A = data_edf.info.items()
#ch_names = A.mapping.get('ch_names')
#nchan = A.mapping.get('nchan') # number of channels
#data_edf.ch_names
#STR = data_edf.get_data()
#df = data_edf.to_data_frame()